In [1]:
#Download ollama
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess
process = subprocess.Popen("ollama serve", shell=True) #runs on a different thread
#Download model
!ollama pull llama3.1:8b
!ollama pull nomic-embed-text
!pip install ollama
import ollama

# Document loading, retrieval methods and text splitting
!pip install -qU langchain langchain_community
!pip install -qU unstructured unstructured-inference pdfminer.six pi_heif
!apt-get -y install poppler-utils

# Local vector store via Chroma
!pip install -qU langchain_chroma

# Local inference and embeddings via Ollama
!pip install -qU langchain_ollama

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%#####                                                               16.4%###                                                         24.3%##########                     74.5%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


2024/09/22 14:00:41 routes.go:1153: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES: http_proxy: https_proxy: no_proxy:]"
time=2024-09-22T14:00:41.625Z level=INFO source=images.go:753 msg="total blobs: 9"
time=2024-09-22T14:00:41.6

[GIN] 2024/09/22 - 14:01:00 | 200 |       73.57µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ 

time=2024-09-22T14:01:00.015Z level=INFO source=types.go:107 msg="inference compute" id=GPU-68a02f56-1de0-05bc-e0b3-5f5e9893d2af library=cuda variant=v12 compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"
time=2024-09-22T14:01:00.015Z level=INFO source=types.go:107 msg="inference compute" id=GPU-83b169cb-2f8e-16fb-b18b-a19af2ecce44 library=cuda variant=v12 compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"


pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ [GIN] 2024/09/22 - 14:01:00 | 200 |  529.838507ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 1a4c3c319823... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 
[GIN] 2024/09/22 - 14:01:01 | 200 |      26.794µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ [GIN] 2024/09/22 - 14:01:02 | 200 |  681.411242ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB   

In [ ]:
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = OnlinePDFLoader("https://www1.hkexnews.hk/listedco/listconews/sehk/2023/0926/2023092600881.pdf")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [ ]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [ ]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.1:8b",
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Group the shareholders and denote their number of shares in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "Who are the shareholders and how many number of shares do they own?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)